In [2]:
### import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyathena import connect
from pyathena.pandas_cursor import PandasCursor

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import xgboost as xgb 

In [3]:
# Define connection to DB
conn = connect(
    s3_staging_dir='s3://aws-athena-query-results-323906537337-us-east-1/',
    region_name='us-east-1',
    cursor_class=PandasCursor
    )
cursor = conn.cursor()

## 0. Data

### 0.1 game data

In [4]:
# query team game-level stats

simple_query = f'''
select 
       cast(season as integer) season,
       teamid,
       eventmetadata.week as week,
       eventmetadata.gameCode as gamecode,
       gametimeinseconds,
       timeofpossessioninseconds,
       totalplaysonfield,
       totaloffensiveplays,
       totaloffensiveplays * 3600 / gametimeinseconds as totaloffensiveplays_normalized,
       totaldesignedpassplays as totaldesignedpassplays,
       totalpassattempts as totalpassattempts,
       -- totalexpectedpassingplays, -- no value
       -- passingpercentage,
       totalrushingattempts,
       totalsacksallowed,
       totalthrowawaysandspikes,
       totalscrambles
       -- totalsacks,
       -- timeofpossessionpergameinminutes, -- normalized into a 60-min game?
       -- timeofpossessioninminutes,  
from datalakefootball.team_aggregated_game_stats
where 
    eventmetadata.week is not null
    and eventmetadata.eventtypeid < 3
order by season, teamid, week
'''

if True:
    game_df = cursor.execute(simple_query).as_pandas()
    print(game_df.info())
else:
    print("Failed to query!")
    
# totaloffensiveplays = totalrushingattempts + totalpassattempts + totalsacksallowed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   season                          4250 non-null   Int64  
 1   teamid                          4250 non-null   Int64  
 2   week                            4250 non-null   Int64  
 3   gamecode                        4250 non-null   Int64  
 4   gametimeinseconds               4250 non-null   float64
 5   timeofpossessioninseconds       4250 non-null   float64
 6   totalplaysonfield               4250 non-null   Int64  
 7   totaloffensiveplays             4250 non-null   Int64  
 8   totaloffensiveplays_normalized  4250 non-null   float64
 9   totaldesignedpassplays          4250 non-null   Int64  
 10  totalpassattempts               4250 non-null   Int64  
 11  totalrushingattempts            4250 non-null   Int64  
 12  totalsacksallowed               42

In [5]:
# from datalake football to query game level information
# query team game-level stats

simple_query = f'''
-- t1 and t2: ceate one record per game from pbp data
with t1 as
(
select
  ROW_NUMBER() OVER(partition by eventmetadata.gamecode) AS num_row, 
  season, eventmetadata, teammetadata
from datalakefootball.pbp
where leagueid='8'
),

t2 as
(
select
  season,
  cast(FROM_UNIXTIME(eventmetadata.gamedateutcepoch) as DATE) date,
  eventmetadata.week, 
  eventmetadata.gamecode game_code, eventmetadata.eventtypeid as event_type_id,
  teammetadata[1].teamid home_team_id, 
  teammetadata[1].abbreviation team1,
  teammetadata[1].score home_team_score,
  teammetadata[2].teamid away_team_id, 
  teammetadata[2].abbreviation team2,
  teammetadata[2].score away_team_score
from t1
where num_row=1
order by game_code
),

-- t3 and t4 are created to decide previous game week for each team
t3 as
(
select
  *,
  ROW_NUMBER() OVER(partition by season, team_id order by week) AS num_row
from
  (select * from (select season, home_team_id as team_id, week from t2)
   union
   select * from (select season, away_team_id as team_id, week from t2))
),

t4 as
(
select c.season, c.team_id, c.week, p.week prev_week
from t3 as c
  left join t3 as p
  on c.season=p.season and c.team_id=p.team_id and c.num_row=p.num_row+1
 order by c.season, c.team_id, c.week
)

-- final game level table
select 
  t2.*, 
  COALESCE(home_table.prev_week,0) as home_team_prev_week,
  COALESCE(away_table.prev_week,0) as away_team_prev_week
from t2 
  left join t4 as home_table
  on t2.season=home_table.season and t2.home_team_id=home_table.team_id and t2.week=home_table.week
  left join t4 as away_table
  on t2.season=away_table.season and t2.away_team_id=away_table.team_id and t2.week=away_table.week
order by t2.season, t2.week
'''

if True:
    game_df = cursor.execute(simple_query).as_pandas()
    print(game_df.info())
else:
    print("Failed to query!")
    
# totaloffensiveplays = totalrushingattempts + totalpassattempts + totalsacksallowed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8492 entries, 0 to 8491
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   season               8492 non-null   object        
 1   date                 8492 non-null   datetime64[ns]
 2   week                 8492 non-null   Int64         
 3   game_code            8492 non-null   Int64         
 4   event_type_id        8492 non-null   Int64         
 5   home_team_id         8492 non-null   Int64         
 6   team1                8492 non-null   object        
 7   home_team_score      8492 non-null   Int64         
 8   away_team_id         8492 non-null   Int64         
 9   team2                8492 non-null   object        
 10  away_team_score      8492 non-null   Int64         
 11  home_team_prev_week  8492 non-null   Int64         
 12  away_team_prev_week  8492 non-null   Int64         
dtypes: Int64(9), datetime64[ns](1), o

### 0.2 expected game data

#### 0.2.2 expected total plays: version 1

In [ ]:
# load expected game data of version 1

simple_query = f'''
select 
   cast(season as integer) season, 
   team.teamid as teamid,
   opponentteamid,
   eventmetadata.week as week,
   totalplays as exp_totalPlays
   -- passingplays as exp_passingPlays, -- no values
   -- rushingplays as exp_rushingPlays
from datalakefootball.team_expected_rates
where 
    season >= '2015'
    and version = '1' 
    and (eventmetadata.eventtypeid = 1 or eventmetadata.eventtypeid = 2)
order by season, teamid, week
'''

if True:
    exp_df = cursor.execute(simple_query).as_pandas()
    print(exp_df.info())
    #plt.plot(exp_df.exp_totalPlays)
else:
    print("Failed to query!")

# merge expected game info
game_df = pd.merge(game_df, exp_df, on=['season','teamid','week'], how='inner')
print(game_df.info())

### 0.3 ytd data (pre-game)

In [ ]:
simple_query = f'''
select
    cast(season as integer) season,
    teamid,
    eventmetadata.week as week, 
    gametimeinseconds as ytd_gameTime,
    totaloffensiveplays as ytd_totalPlays,
    totaldesignedpassplays / floor(gametimeinseconds / 3600) as ytd_passPlaysPerGame, -- overtime is estimated as well
    pace as ytd_pace,
    timeofpossessioninseconds as ytd_TOP,
    timeofpossessioninseconds / floor(gametimeinseconds / 3600) as ytd_TOPperGame, -- consider ot
    passingpercentage as ytd_passingpercentage,

    totalpoints / floor(gametimeinseconds / 3600) as ytd_totalPointsPerGame,  -- consider ot
    totalsacks / floor(gametimeinseconds / 3600) as ytd_totalSacksPerGame     -- consider ot
    
from datalakefootball.team_aggregated_ytd_stats
where 
    season >= '2013' 
    and (eventmetadata.eventtypeid = 1 or eventmetadata.eventtypeid = 2)
    and eventmetadata.week is not null
order by season, teamid, week
'''

if True:
    ytd_df = cursor.execute(simple_query).as_pandas()
    print(ytd_df.info())
else:
    print("Failed to query!")
    
ytd_df['ytd_offensivePlaysPerGame'] = ytd_df.ytd_TOPperGame / ytd_df.ytd_pace
    
# merge ytd data into game_data
game_df = pd.merge(game_df, ytd_df, on=['season','teamid','week'], how='left')

In [ ]:
simple_query = f'''
select
    cast(season as integer) season,
    teamid,
    eventmetadata.week as week, 
    pace as ytd_pace_conceded
from datalakefootball.team_aggregated_ytd_stats_conceded
where 
    season >= '2013' 
    and (eventmetadata.eventtypeid = 1 or eventmetadata.eventtypeid = 2)
    and eventmetadata.week is not null
order by season, teamid, week
'''
if True:
    ytd_df_conceded = cursor.execute(simple_query).as_pandas()
    print(ytd_df_conceded.info())
else:
    print("Failed to query!")
    
game_df = pd.merge(game_df, ytd_df_conceded, on=['season','teamid','week'], how='left')
game_df.info()


# add opponent ytd_data
tmp = game_df[['season','teamid','week','ytd_pace','ytd_pace_conceded','ytd_offensivePlaysPerGame']].copy()
tmp.rename(columns={'teamid':'opponentteamid', 
                    'ytd_pace':'ytd_o_pace', 
                    'ytd_pace_conceded':'ytd_o_pace_conceded',
                    'ytd_offensivePlaysPerGame':'ytd_o_offensivePlaysPerGame'}, inplace=True)
game_df = pd.merge(game_df, tmp, on=['season','opponentteamid','week'], how='left')
game_df.shape

#### 0.3.1 ytd_yards_per_rush / passing

In [ ]:
simple_query = f'''
select cast(season as integer) season, 
       teamid, eventmetadata.gameCode gamecode, eventmetadata.week week, 
       passing.attempts passingAttempts, 
       passing.yards passingYards,
       rushing.attempts rushingAttempts, 
       rushing.yards rushingYards
from datalakefootball.team_stats_game
where season>='2013' and (eventmetadata.eventtypeid=1 or eventmetadata.eventtypeid=2)
order by season, teamid, week
'''

if True:
    yards_df = cursor.execute(simple_query).as_pandas()
    print(yards_df.info())
else:
    print("Failed to query!")

# merge exp_df into yards_df to give it opponent_team_id

yards_df = pd.merge(yards_df, game_df[['season','teamid','gamecode','opponentteamid']], 
                    on=['season','teamid','gamecode'], how='right')

In [ ]:
# calculate cumulated sum and then yardsPerAttempt
gd = yards_df.groupby(['season','teamid'])

yards_df['ytd_passingYardsPerAttempt'] = gd.passingYards.cumsum() / gd.passingAttempts.cumsum()
yards_df['ytd_rushingYardsPerAttempt'] = gd.rushingYards.cumsum() / gd.rushingAttempts.cumsum()

yards_df['ytd_passingYardsPerAttempt'] = gd.ytd_passingYardsPerAttempt.shift(1)
yards_df['ytd_rushingYardsPerAttempt'] = gd.ytd_rushingYardsPerAttempt.shift(1)

# calculate opponent cumulated sum and then yardsPerAttempt
yards_df.sort_values(['season','opponentteamid','week'], inplace=True)

gd = yards_df.groupby(['season','opponentteamid'])

yards_df['ytd_o_passingYardsPerAttempt_conceded'] = gd.passingYards.cumsum() / gd.passingAttempts.cumsum()
yards_df['ytd_o_rushingYardsPerAttempt_conceded'] = gd.rushingYards.cumsum() / gd.rushingAttempts.cumsum()

yards_df['ytd_o_passingYardsPerAttempt_conceded'] = gd.ytd_o_passingYardsPerAttempt_conceded.shift(1)
yards_df['ytd_o_rushingYardsPerAttempt_conceded'] = gd.ytd_o_rushingYardsPerAttempt_conceded.shift(1)

In [ ]:
# merge to get team conceded yards
tmp = yards_df[['season', 'opponentteamid', 'week',
                       'ytd_o_passingYardsPerAttempt_conceded','ytd_o_rushingYardsPerAttempt_conceded']]

tmp.rename(columns={'opponentteamid':'teamid',
                      'ytd_o_passingYardsPerAttempt_conceded':'ytd_passingYardsPerAttempt_conceded',
                      'ytd_o_rushingYardsPerAttempt_conceded':'ytd_rushingYardsPerAttempt_conceded'}, inplace=True)

yards_df = pd.merge(yards_df, tmp, on=['season', 'teamid', 'week'], how='left')

# create opponents passing/rushing yards
tmp = yards_df[['season','teamid','week','ytd_passingYardsPerAttempt','ytd_rushingYardsPerAttempt']]

tmp.rename(columns={'teamid':'opponentteamid',
                        'ytd_passingYardsPerAttempt':'ytd_o_passingYardsPerAttempt',
                        'ytd_rushingYardsPerAttempt':'ytd_o_rushingYardsPerAttempt'}, inplace=True)

yards_df = pd.merge(yards_df, tmp, on = ['season','week','opponentteamid'], how='left')

yards_df.info()

In [ ]:
# merge yards info into game_df

print(game_df.shape)
game_df = pd.merge(game_df, 
                   yards_df[['season','teamid','week',
                             'ytd_passingYardsPerAttempt','ytd_rushingYardsPerAttempt',
                             'ytd_passingYardsPerAttempt_conceded', 'ytd_rushingYardsPerAttempt_conceded',
                             'ytd_o_passingYardsPerAttempt','ytd_o_rushingYardsPerAttempt',
                             'ytd_o_passingYardsPerAttempt_conceded', 'ytd_o_rushingYardsPerAttempt_conceded']],
                  on = ['season','teamid','week'], how='left')
print(game_df.shape)

#### 0.3.2 ytd_scrambles

In [ ]:
simple_query = f'''
select cast(season as integer) season, 
        teamid, eventmetadata.week week,
       sum(totalscrambles) game_scrambles, 1 idx
from datalakefootball.player_aggregated_game_stats
where season >= '2013' and leagueid='8'
group by season, teamid, eventmetadata.week
order by season, teamid, week
'''

if True:
    scrambles_df = cursor.execute(simple_query).as_pandas()
    print(scrambles_df.info())
else:
    print("Failed to query!")
    
    
# calculate cumulated sum and then yardsPerAttempt
gd = scrambles_df.groupby(['season','teamid'])

scrambles_df['ytd_scrambles'] = gd.game_scrambles.cumsum() / gd.idx.cumsum()

scrambles_df['ytd_scrambles'] = gd.ytd_scrambles.shift(1)


# merge scrambles into game_df
game_df = pd.merge(game_df, scrambles_df[['season','teamid','week','ytd_scrambles']], 
                   on=['season','teamid','week'], how='left')

game_df['ytd_scrambleRatio'] = game_df.ytd_scrambles / game_df.ytd_passPlaysPerGame
game_df.shape

### 0.4 Prepare baseline data

In [ ]:
# create baseline case from ytd data

baseline_df = game_df[['season','teamid','ytd_gameTime','ytd_pace','ytd_pace_conceded',
                        'ytd_passingpercentage','ytd_offensivePlaysPerGame','ytd_passPlaysPerGame','ytd_TOPperGame']]
baseline_df = baseline_df.groupby(['season','teamid']).tail(1)

baseline_df.rename(columns={'ytd_gameTime':'base_gameTime',
                              'ytd_pace':'base_pace', 
                              'ytd_pace_conceded':'base_pace_conceded',
                              'ytd_offensivePlaysPerGame':'base_offensivePlaysPerGame',
                              'ytd_passPlaysPerGame':'base_passPlaysPerGame',
                              'ytd_passingpercentage':'base_passingpercentage',
                              'ytd_TOPperGame':'base_TOPperGame'},
                                inplace=True)

baseline_df.season = baseline_df.season + 1

# merge baseline info into game_df, in this case, we will lose 2013
game_df = pd.merge(game_df, baseline_df, on=['season','teamid'], how='inner')
game_df.shape

In [ ]:
# compute regressed YTD team passing %, using ytd and baseline info
# compute expected passing play
game_df['reg_passingpercentage'] = (350 * game_df.base_passingpercentage + 
                                    game_df.ytd_totalPlays * game_df.ytd_passingpercentage) / \
                                   (350 + game_df.ytd_totalPlays)
game_df['exp_passingPlays'] = game_df.exp_totalPlays * game_df.reg_passingpercentage

#plt.plot(game_df.exp_passingPlays[game_df.week > 5])

In [ ]:
# add baseline info for opponent team

tmp = game_df[['teamid','gamecode','ytd_TOPperGame','ytd_totalPointsPerGame','ytd_totalSacksPerGame',
               'base_pace','base_pace_conceded','base_offensivePlaysPerGame','base_TOPperGame']].copy()
tmp.rename(columns={'teamid':'opponentteamid',
                    'ytd_TOPperGame':'ytd_o_TOPperGame',
                    'ytd_totalPointsPerGame':'ytd_o_totalPointsPerGame',
                    'ytd_totalSacksPerGame':'ytd_o_totalSacksPerGame',
                    'base_pace':'base_o_pace',
                    'base_pace_conceded':'base_o_pace_conceded',
                    'base_offensivePlaysPerGame':'base_o_offensivePlaysPerGame',
                    'base_passingpercentage':'base_o_passingpercentage',
                    'base_TOPperGame':'base_o_TOPperGame'}, 
                       inplace=True)

game_df = pd.merge(game_df, tmp, on = ['opponentteamid', 'gamecode'], how='left')
game_df.shape

### 0.5 Pre-game odds

In [ ]:
simple_query = f'''
WITH mainQuery AS
(
    WITH subQuery AS
    (
        SELECT
            event.eventId AS eventId,
            lineList.scope.name AS lineScope,
            lineList.line AS lineData,
            cast(season as integer) season,
            eventmetadata.gamecode gamecode
        FROM datalakefootball.odds,
            UNNEST(event.lines) t(lineList)
        WHERE leagueid='8'
    )
    SELECT
        season,
        gamecode,
        eventId,
        lineScope,
        sublineList.lineType.name AS lineType,
        sublineList.total AS overUnderPoints,
        sublineList.favoritePoints,
        sublineList.favoriteTeamId,
        IF(sublineList.favoriteMoney='EVEN', 100.0, 1.0 * CAST(sublineList.favoriteMoney AS INTEGER)) AS favoriteMoney,
        IF(sublineList.underdogMoney='EVEN', 100.0, 1.0 * CAST(sublineList.underdogMoney AS INTEGER)) AS underdogMoney,
        IF(sublineList.homeMoney='EVEN', 100.0, 1.0 * CAST(sublineList.homeMoney AS INTEGER)) AS homeMoney,
        IF(sublineList.awayMoney='EVEN', 100.0, 1.0 * CAST(sublineList.awayMoney AS INTEGER)) AS awayMoney,
        IF(sublineList.overMoney='EVEN', 100.0, 1.0 * CAST(sublineList.overMoney AS INTEGER)) AS overMoney,
        IF(sublineList.underMoney='EVEN', 100.0, 1.0 * CAST(sublineList.underMoney AS INTEGER)) AS underMoney
    FROM subQuery,
        UNNEST(lineData) t(sublineList)
)
SELECT
    season,
    gamecode, -- same as eventId
    -- eventId,
    -- lineScope,
    -- lineType,
    overUnderPoints,
    favoritePoints,
    favoriteTeamId,
    -- favoriteMoney,
    CASE
        WHEN favoriteMoney < 0 THEN - (100 - favoriteMoney) / favoriteMoney
        WHEN favoriteMoney > 0 THEN (100 + favoriteMoney) / 100
    END AS favoriteMoneyDecimal,
    -- underdogMoney,
    CASE
        WHEN underdogMoney < 0 THEN - (100 - underdogMoney) / underdogMoney
        WHEN underdogMoney > 0 THEN (100 + underdogMoney) / 100
    END AS underdogMoneyDecimal,
    -- homeMoney,
    CASE
        WHEN homeMoney < 0 THEN - (100 - homeMoney) / homeMoney
        WHEN homeMoney > 0 THEN (100 + homeMoney) / 100
    END AS homeMoneyDecimal,
    -- awayMoney,
    CASE
        WHEN awayMoney < 0 THEN - (100 - awayMoney) / awayMoney
        WHEN awayMoney > 0 THEN (100 + awayMoney) / 100
    END AS awayMoneyDecimal,
    -- overMoney,
    CASE
        WHEN overMoney < 0 THEN - (100 - overMoney) / overMoney
        WHEN overMoney > 0 THEN (100 + overMoney) / 100
    END AS overMoneyDecimal,
    -- underMoney,
    CASE
        WHEN underMoney < 0 THEN - (100 - underMoney) / underMoney
        WHEN underMoney > 0 THEN (100 + underMoney) / 100
    END AS underMoneyDecimal
FROM mainQuery
WHERE lineType = 'current' and season >= 2016
order by season, gamecode
'''

if True:
    odds_df = cursor.execute(simple_query).as_pandas()
    print(odds_df.info())
else:
    print("Failed to query!")

In [ ]:
# merge odds into game_df

tmp = game_df.loc[game_df.season >= 2016, ['gamecode','teamid','opponentteamid']].copy()

tmp2 = odds_df[['gamecode','favoriteTeamId','homeMoneyDecimal','awayMoneyDecimal','favoritePoints']].copy()
tmp2.loc[:,'odds'] = odds_df.homeMoneyDecimal / odds_df.awayMoneyDecimal

tmp2 = pd.merge(tmp, tmp2[['gamecode','favoriteTeamId','odds','favoritePoints']], on=['gamecode'], how='left')

id = tmp2.teamid == tmp2.favoriteTeamId
tmp2.loc[id,'odds'] = 1 / tmp2.odds[id]

tmp2.loc[id,  'favoritePoints'] = abs(tmp2.favoritePoints[id])
tmp2.loc[~id, 'favoritePoints'] = -abs(tmp2.favoritePoints[~id])

#tmp2.sort_values(['gamecode'])

In [ ]:
# merge odds df into game_df
game_df = pd.merge(game_df, tmp2[['gamecode','teamid','odds','favoritePoints']], on=['gamecode','teamid'], how='left')
game_df.shape

## 1. Featuring preparation

In [ ]:
# we may want to re-calculate the expected total plays given the data issue with the 1st 5 weeks

### 1.1 weighted historical values

In [ ]:
# alpha controls the weighting factor calculation

alpha = 2.0

w = game_df.ytd_gameTime * alpha / (game_df.ytd_gameTime * alpha + game_df.base_gameTime)

game_df['ytd_offensivePlaysPerGameAdj'] = game_df.ytd_offensivePlaysPerGame * w + game_df.base_offensivePlaysPerGame * (1-w)

game_df['ytd_passPlaysPerGameAdj'] = game_df.ytd_passPlaysPerGame * w + game_df.base_passPlaysPerGame * (1-w)

game_df['ytd_passRatioAdj'] = game_df['ytd_passPlaysPerGameAdj'] / game_df['ytd_offensivePlaysPerGameAdj']

w_gameTime = (game_df.ytd_TOPperGame + game_df.ytd_o_TOPperGame) * w + \
                (game_df.base_TOPperGame + game_df.base_o_TOPperGame) * (1-w)

# do we need this game time adjustment????
#game_df['ytd_offensivePlaysAdj'] = 3600 / w_gameTime * game_df['ytd_offensivePlaysPerGameAdj']    
game_df['ytd_offensivePlaysAdj'] = game_df['ytd_offensivePlaysPerGameAdj']

### 1.2 team pace adjustments

In [ ]:
w_pace = game_df.ytd_pace * w + game_df.base_pace * (1-w)
w_pace_conceded = game_df.ytd_pace_conceded * w + game_df.base_pace_conceded * (1-w)

w_o_pace = game_df.ytd_o_pace * w + game_df.base_o_pace * (1-w)
w_o_pace_conceded = game_df.ytd_o_pace_conceded * w + game_df.base_o_pace_conceded * (1-w)

# adjustment term by pace

game_df['ytd_paceConcededAdj'] = w_o_pace - w_pace_conceded
 
game_df['ytd_paceAdj'] = w_pace - w_o_pace_conceded  

### 1.3 match-up specific historical TPPP

In [ ]:
game_df['ytd_passingYardsAdj'] = game_df.ytd_passingYardsPerAttempt - game_df.ytd_o_passingYardsPerAttempt
game_df['ytd_rushingYardsAdj'] = game_df.ytd_rushingYardsPerAttempt - game_df.ytd_o_rushingYardsPerAttempt

game_df['ytd_passingYardsAdj2'] = game_df.ytd_passingYardsPerAttempt - game_df.ytd_o_passingYardsPerAttempt_conceded
game_df['ytd_rushingYardsAdj2'] = game_df.ytd_rushingYardsPerAttempt - game_df.ytd_o_rushingYardsPerAttempt_conceded

In [ ]:
game_df.info()

### 1.4 Feature transformation

### (1) Features for TPPG

In [ ]:
#for total plays
num_fields = ['ytd_offensivePlaysAdj', 
              'ytd_paceAdj', 
              'ytd_paceConcededAdj', 
             ]

features_TPPG = game_df[num_fields]

id = ~features_TPPG.isna().any(axis=1)
features_TPPG = features_TPPG[id]
print(features_TPPG.shape)

id_train = (game_df.season[id] <= 2018).tolist()
id_test  = (game_df.season[id] == 2019).tolist()
print(len(id_train), len(id_test))

label_TPPG = game_df[id].totaloffensiveplays.astype(float)

transform_pipeline = ColumnTransformer(transformers=[
                                            ('num', StandardScaler(), num_fields),
                                        ])

features_TPPG = transform_pipeline.fit_transform(features_TPPG)


feature_names = num_fields.copy()
#cat_one_hot_fields = transform_pipeline.named_transformers_.cat.get_feature_names(input_features=cat_fields)
#feature_names.extend(cat_one_hot_fields)
print("Features:", feature_names)

if type(features_TPPG) == np.ndarray:
    features_TPPG = pd.DataFrame(features_TPPG, columns=feature_names)
else:
    features_TPPG = pd.DataFrame(features_TPPG.toarray(), columns=feature_names)
    

# split data into train(2014~2018), test(2019)
features_TPPG_train = features_TPPG[id_train]
features_TPPG_test  = features_TPPG[id_test]

label_TPPG_train = label_TPPG[id_train]
label_TPPG_test  = label_TPPG[id_test]

print(features_TPPG_train.shape, label_TPPG_train.shape, features_TPPG_test.shape, label_TPPG_test.shape)

### (2) Features for team pass ratio

In [ ]:
# for pass ratio
num_fields = ['ytd_passRatioAdj',
              'ytd_scrambles',
              'ytd_scrambleRatio',
             #'odds',
             'favoritePoints',
             'ytd_passingYardsPerAttempt',
             'ytd_rushingYardsPerAttempt',
             'ytd_o_passingYardsPerAttempt',
             'ytd_o_rushingYardsPerAttempt',
             'ytd_passingYardsPerAttempt_conceded',
             'ytd_rushingYardsPerAttempt_conceded',
             'ytd_o_passingYardsPerAttempt_conceded',
             'ytd_o_rushingYardsPerAttempt_conceded',
             ]
              
features_pass_ratio = game_df[num_fields]

print(features_pass_ratio.shape)
id = ~features_pass_ratio.isna().any(axis=1)
features_pass_ratio = features_pass_ratio[id]
print(features_pass_ratio.shape)

id_train = (game_df.season[id] <= 2018).tolist()
id_test  = (game_df.season[id] == 2019).tolist()
print(len(id_train), len(id_test))

label_pass_ratio = (game_df[id].totaldesignedpassplays.astype(float) / game_df[id].totaloffensiveplays).astype(float)

transform_pipeline = ColumnTransformer(transformers=[
                                            ('num', StandardScaler(), num_fields),
                                        ])

features_pass_ratio = transform_pipeline.fit_transform(features_pass_ratio)

feature_names = num_fields.copy()
#cat_one_hot_fields = transform_pipeline.named_transformers_.cat.get_feature_names(input_features=cat_fields)
#feature_names.extend(cat_one_hot_fields)
print("Features:", feature_names)

if type(features_pass_ratio) == np.ndarray:
    features_pass_ratio = pd.DataFrame(features_pass_ratio, columns=feature_names)
else:
    features_pass_ratio = pd.DataFrame(features_pass_ratio.toarray(), columns=feature_names)
    

# split data into train(2014~2018), test(2019)
features_pass_ratio_train = features_pass_ratio[id_train]
features_pass_ratio_test  = features_pass_ratio[id_test]

label_pass_ratio_train = label_pass_ratio[id_train]
label_pass_ratio_test  = label_pass_ratio[id_test]

print(features_pass_ratio_train.shape, label_pass_ratio_train.shape, 
      features_pass_ratio_test.shape, label_pass_ratio_test.shape)


## 2. Model study

In [ ]:
MAE = make_scorer(mean_absolute_error)
folds = 5

### 2.1 Team expected total plays

### 2.1.1 simple ytd model vs. benchmark model

In [ ]:
id = (game_df.season <= 2018) & (game_df.week > 5)

mse_1 = np.abs((game_df.totaloffensiveplays - game_df.ytd_offensivePlaysPerGame)[id]).mean()

mse_2 = np.abs((game_df.totaloffensiveplays - game_df.exp_totalPlays)[id]).mean()

print('ytd_TPPP model MAE:  {:.2f}'.format(mse_1))
print('Benchmark model MAE: {:.2f}'.format(mse_2))

In [ ]:
(game_df.totaloffensiveplays - game_df.exp_totalPlays)[id].describe()

### 2.1.2 simple ytd_n model

In [ ]:
# note, ytd_n calculated this way is not normalized

id = (game_df.season <= 2018)
tmp = game_df[id].copy()

gd = tmp.groupby(by=['season','teamid'])
tmp['ytd_1_offensivePlaysPerGame'] = gd['totaloffensiveplays_normalized'].shift(1).rolling(1).mean()
tmp['ytd_2_offensivePlaysPerGame'] = gd['totaloffensiveplays_normalized'].shift(1).rolling(2).mean()
tmp['ytd_3_offensivePlaysPerGame'] = gd['totaloffensiveplays_normalized'].shift(1).rolling(3).mean()
tmp['ytd_4_offensivePlaysPerGame'] = gd['totaloffensiveplays_normalized'].shift(1).rolling(4).mean()
tmp['ytd_5_offensivePlaysPerGame'] = gd['totaloffensiveplays_normalized'].shift(1).rolling(5).mean()

print('ytd_1 case: {:.2f}'.format(np.abs((tmp.totaloffensiveplays - tmp.ytd_1_offensivePlaysPerGame)).mean()) )
print('ytd_2 case: {:.2f}'.format(np.abs((tmp.totaloffensiveplays - tmp.ytd_2_offensivePlaysPerGame)).mean()) )
print('ytd_3 case: {:.2f}'.format(np.abs((tmp.totaloffensiveplays - tmp.ytd_3_offensivePlaysPerGame)).mean()) )
print('ytd_4 case: {:.2f}'.format(np.abs((tmp.totaloffensiveplays - tmp.ytd_4_offensivePlaysPerGame)).mean()) )
print('ytd_5 case: {:.2f}'.format(np.abs((tmp.totaloffensiveplays - tmp.ytd_5_offensivePlaysPerGame)).mean()) )

### 2.1.3 model performance comparison

In [ ]:
model_linear = SGDRegressor(max_iter=10000, tol=1e-6, random_state=42)
#model_linear = LinearRegression()

MAE_linear = cross_val_score(model_linear, features_TPPG_train, label_TPPG_train, cv=folds, scoring=MAE)
print('Linear regression model MAE: {:.3f}'.format(np.mean(MAE_linear)) )

model_svr = LinearSVR(max_iter=10000, tol=1e-6, random_state=42)
MAE_svr = cross_val_score(model_svr, features_TPPG_train, label_TPPG_train, cv=folds, scoring=MAE)
print('SV regression MAE: {:.3f}'.format(np.mean(MAE_svr)))

model_rf = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=42)
MAE_rf = cross_val_score(model_rf, features_TPPG_train, label_TPPG_train, cv=folds, scoring=MAE)
print('random forest regression MAE: {:.3f}'.format(np.mean(MAE_rf)) )

model_xgb = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.5, learning_rate = 0.1, booster='gbtree',
                max_depth = 5, alpha = 10, n_estimators = 50)
MAE_xgb = cross_val_score(model_xgb, features_TPPG_train, label_TPPG_train, cv=folds, scoring=MAE)
print('XGBoost regression MAE: {:.3f}'.format(np.mean(MAE_xgb)) )

In [ ]:
# linear regression model interpretation

import statsmodels.api as sm

mod = sm.OLS(np.array(label_train), np.array(features_train), missing='drop')
res = mod.fit()

mae = np.abs(res.resid).mean()

print('{:.2f}'.format(mae) )
res.summary()

### 2.2 Team expected passing ratio

In [ ]:
# benchmark case
# there is a mis-match: expected value is normalized

id = (game_df.season <= 2018) & (game_df.week > 5)

mse_1 = np.abs((game_df.totaldesignedpassplays - game_df.ytd_passPlaysPerGame)[id] / game_df.totaloffensiveplays[id]).mean()

mse_2 = np.abs((game_df.totaldesignedpassplays - game_df.ytd_passPlaysPerGameAdj)[id] / game_df.totaloffensiveplays[id]).mean()

mse_3 = np.abs((game_df.totaldesignedpassplays - game_df.exp_passingPlays)[id] / game_df.totaloffensiveplays[id]).mean()

print('average_passRatio: {:.3f}\n'.format(label_pass_ratio_train.mean()))

print('ytd_passRatio model MAE: {:.3f}'.format(mse_1))
print('w_passRatio model MAE:   {:.3f}'.format(mse_2))
print('Benchmark model MAE:     {:.3f}'.format(mse_3))

In [ ]:
model_linear = SGDRegressor(max_iter=10000, tol=1e-6)

MAE_linear = cross_val_score(model_linear, features_pass_ratio_train, label_pass_ratio_train, cv=folds, scoring=MAE)
print('Linear regression model MAE: {:.4f}'.format(np.mean(MAE_linear)) )

model_svr = LinearSVR(max_iter=10000, tol=1e-3)
MAE_svr = cross_val_score(model_svr, features_pass_ratio_train, label_pass_ratio_train, cv=folds, scoring=MAE)
print('SV regression MAE: {:.4f}'.format(np.mean(MAE_svr)))

model_rf = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=0)
MAE_rf = cross_val_score(model_rf, features_pass_ratio_train, label_pass_ratio_train, cv=folds, scoring=MAE)
print('random forest regression MAE: {:.4f}'.format(np.mean(MAE_rf)) )

model_xgb = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.5, learning_rate = 0.1, booster='gbtree',
                max_depth = 5, alpha = 10, n_estimators = 50)
MAE_xgb = cross_val_score(model_xgb, features_pass_ratio_train, label_pass_ratio_train, cv=folds, scoring=MAE)
print('XGBoost regression MAE: {:.4f}'.format(np.mean(MAE_xgb)) )

## 3 Model testing

### 3.1 Team total plays per game

In [ ]:
# baseline case
id = (game_df.season > 2018) & (game_df.week > 5)

mse_1 = np.abs((game_df.totaloffensiveplays - game_df.ytd_offensivePlaysPerGame)[id]).mean()

mse_2 = np.abs((game_df.totaloffensiveplays - game_df.exp_totalPlays)[id]).mean()

print('ytd_TPPP model MAE:  {:.3f}'.format(mse_1))
print('Benchmark model MAE: {:.3f}'.format(mse_2))

In [ ]:
# linear model
model_linear.fit(features_TPPG_train, label_TPPG_train)
mae = np.abs(model_linear.predict(features_TPPG_test) - label_TPPG_test).mean()
print('linear regression MAE: {:.3f}'.format(mae) )

In [ ]:
# svm model
model_svr.fit(features_TPPG_train, label_TPPG_train)
mae = np.abs(model_svr.predict(features_TPPG_test) - label_TPPG_test).mean()
print('SVM regression MAE: {:.3f}'.format(mae) )

In [ ]:
# random forest
model_rf.fit(features_TPPG_train, label_TPPG_train)
mae = np.abs(model_rf.predict(features_TPPG_test) - label_TPPG_test).mean()
print('random forest regression MAE: {:.3f}'.format(mae) )

In [ ]:
# gradient boost
model_xgb.fit(features_TPPG_train, label_TPPG_train)

label_hat = model_xgb.predict(features_TPPG_test)
mae = np.abs(label_hat - np.array(label_TPPG_test)).mean()

print('gradient boost regression MAE: {:.3f}'.format(mae) )

### 3.2 Pass Ratio

In [ ]:
# baseline case
id = (game_df.season > 2018) & (game_df.week > 5)

mse_1 = np.abs((game_df.totaldesignedpassplays - game_df.ytd_passPlaysPerGame)[id] / game_df.totaloffensiveplays[id]).mean()
mse_2 = np.abs((game_df.totaldesignedpassplays - game_df.ytd_passPlaysPerGameAdj)[id] / game_df.totaloffensiveplays[id]).mean()
mse_3 = np.abs((game_df.totaldesignedpassplays - game_df.exp_passingPlays)[id] / game_df.totaloffensiveplays[id]).mean()

print('average_passRatio: {:.3f}\n'.format(label_train.mean()))

#print('ytd_passRatio model MAE: {:.3f}'.format(mse_1))
#print('w_passRatio model MAE:   {:.3f}'.format(mse_2))
print('Benchmark model MAE:          {:.3f}'.format(mse_3))

MAE_linear = cross_val_score(model_linear, features_test_s, label_test, cv=folds, scoring=MAE)
print('Linear regression model MAE:  {:.3f}'.format(np.mean(MAE_linear)) )

MAE_svr = cross_val_score(model_svr, features_test_s, label_test, cv=folds, scoring=MAE)
print('SV regression MAE:            {:.3f}'.format(np.mean(MAE_svr)))

MAE_rf = cross_val_score(model_rf, features_test, label_test, cv=folds, scoring=MAE)
print('random forest regression MAE: {:.3f}'.format(np.mean(MAE_rf)) )

MAE_xgb = cross_val_score(model_xgb, features_test, label_test, cv=folds, scoring=MAE)
print('XGBoost regression MAE:       {:.3f}'.format(np.mean(MAE_xgb)) )